<a href="https://colab.research.google.com/github/ticklecatisback/My-colab-Collections/blob/main/SimSwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a simple example of SimSwap on processing video with multiple faces. You can change the codes for inference based on our other scripts for image or single face swapping.

* Code path: https://github.com/neuralchen/SimSwap

* Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

* Original notebook: https://colab.research.google.com/github/neuralchen/SimSwap/blob/main/SimSwap%20colab.ipynb



In [ ]:
#@title Check CPU
#@markdown make sure you are using a runtime with GPU.
#@markdown you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

In [ ]:
#@title Setup
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")
 
from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')

!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1
!pip install -U youtube-dl
!pip install imageio-ffmpeg==0.4.5
import os
os.chdir("SimSwap")
!ls

from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

### Now onedrive file can be downloaded in Colab directly!
### If the link blow is not permanent, you can just download it from the 
### open url(can be found at [our repo]/doc/guidance/preparation.md) and copy the assigned download link here.
### many thanks to woctezuma for this very useful help
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/

from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

In [ ]:
#@title Imports
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [ ]:
#@title STEP2: Select a Youtube Video
video_url = 'https://www.youtube.com/watch?v=vAnWYLTdvfY' #@param {type:"string"}

import shutil

import os
import os.path
from IPython.display import clear_output
import youtube_dl

loadingAn(name="lds")
textAn("Downloading Video...", ty='twg')
if video_url:
  !rm -f /content/SimSwap/demo_file/youtube.mp4
  !youtube-dl -f "bestvideo[ext=mp4][vcodec!*=av01][height<=360]+bestaudio[ext=m4a]/mp4[height<=360][vcodec!*=av01]/mp4[vcodec!*=av01]/mp4" "$video_url" --merge-output-format mp4 -o /content/SimSwap/demo_file/youtube.mp4

# cut the video
textAn("Trimming Video...", ty='twg')

start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)

if duration_seconds:
  !mv /content/SimSwap/demo_file/youtube.mp4 /content/SimSwap/demo_file/full_video.mp4
  !ffmpeg -ss $start_seconds -t $duration_seconds -i /content/full_video.mp4 -f mp4 /content/SimSwap/demo_file/youtube.mp4 -y

!rm -df youtube.mp4
# download the youtube with the given ID

#delete video.mp4 if already exits
!rm -f /content/SimSwap/demo_file/ass.mp4


#Preview trimmed video
clear_output()
print("Trimmed Video")
showVideo('/content/SimSwap/demo_file/youtube.mp4')

In [ ]:
#@title STEP2: Upload your Video File
from google.colab import files
import shutil
uploaded = files.upload()
dst ='input_vid.mp4' 
os.rename(list(uploaded.keys())[0], dst)

loadingAn(name="lds")
textAn("Moving Video...", ty='twg')

PATH_TO_YOU_VIDEO = '/content/input_vid.mp4'
import os, sys, re
import subprocess

shutil.move("/content/SimSwap/input_vid.mp4", "/content/SimSwap/demo_file") 
#@markdown ### Trim the video (start, end) seconds
#@markdown <i>Don't want to trim ? put <b>'start'</b> = -1 and <b>'end'</b> = -1</i>
start =  -1#@param {type:"integer"}
end =  -1#@param {type:"integer"}


#@markdown <i>Note: the trimmed video must have face on all frames</i>

# delete start end
interval = end - start

textAn("Triming Video...", ty='twg')

#delete if file already exists
!rm -f '/content/input_fi.mp4'

if start < 0 or end < 0:
  #convert the video to specif location
  !ffmpeg -i '$PATH_TO_YOU_VIDEO' '/content/SimSwap/demo_file/input_vid.mp4'
else:
  # cut the video
  !ffmpeg -i '$PATH_TO_YOU_VIDEO' -ss {start} -t {interval} -async 1 '/content/SimSwap/demo_file/input_vid.mp4'
from IPython.display import clear_output

# Preview trimmed video
clear_output()
print("Input Video")
showVideo('/content/SimSwap/demo_file/input_vid.mp4')

In [ ]:
#@title Delete old input Video file
%rm /content/SimSwap/demo_file/input_vid.mp4
from IPython.display import clear_output 
clear_output()

In [ ]:
#@title STEP2: Select a Image
image_url = 'https://media.bizj.us/view/img/11965652/glennfukuda-4196c-pr*1500xx1819-2425-61-0.jpg' #@param {type:"string"}
import shutil
import cv2
from IPython.display import Image

if image_url:
  !wget "$image_url" -O /content/SimSwap/demo_file/image.png

Image('/content/SimSwap/demo_file/image.png')

In [ ]:
#@title STEP2+: Upload your Image
import os
from google.colab import files
import cv2
import shutil
from IPython.display import Image
uploaded = files.upload()
dst ='image.png' 
os.rename(list(uploaded.keys())[0], dst)
shutil.move("/content/SimSwap/image.png", "/content/SimSwap/demo_file")

Image('/content/SimSwap/demo_file/image.png')

In [ ]:
#@title Delete old input Image file
%rm /content/SimSwap/demo_file/image.png
from IPython.display import clear_output 
clear_output()

In [20]:
#@title Compose
transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [ ]:
#@title Training
training = False #@param {type:"boolean"}
mask = True #@param {type:"boolean"}
import tensorflow as tf
import numpy as np
from PIL import Image
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.pic_a_path = '/content/SimSwap/demo_file/image.png'
opt.video_path = '/content/SimSwap/demo_file/input_vid.mp4' ## or replace it with video from your own google drive
opt.output_path = '/content/SimSwap/output/demo.mp4'
opt.temp_path = './tmp'
opt.Arc_path = '/content/SimSwap/arcface_model/arcface_checkpoint.tar'
opt.isTrain = False
opt.use_mask = False  ## new feature up-to-date

crop_size = opt.crop_size

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

loadingAn(name="lds")
textAn("Analyzing Video...", ty='twg')

with torch.no_grad():
    pic_a = opt.pic_a_path
    # img_a = Image.open(pic_a).convert('RGB')
    img_a_whole = cv2.imread(pic_a)
    img_a_align_crop, _ = app.get(img_a_whole,crop_size)
    img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB)) 
    img_a = transformer_Arcface(img_a_align_crop_pil)
    img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

    # convert numpy to tensor
    img_id = img_id.cuda()

    #create latent id
    img_id_downsample = F.interpolate(img_id, size=(112,112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = latend_id.detach().to('cpu')
    latend_id = latend_id/np.linalg.norm(latend_id,axis=1,keepdims=True)
    latend_id = latend_id.to('cuda')

    video_swap(opt.video_path, latend_id, model, app, opt.output_path, temp_results_dir=opt.temp_path, use_mask=opt.use_mask)

clear_output()
print("Dowload this video from", "/content/SimSwap/output/demo.mp4")
showVideo('/content/SimSwap/output/demo.mp4')

In [ ]:
#@title Download Video
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "output.mp4", "combined.mp4" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('/content/SimSwap/output/demo.mp4') #fails for Firefox private window

In [ ]:
#@title Delete old result file
%rm /content/SimSwap/output/demo.mp4
from IPython.display import clear_output 
clear_output()

# Image face swap coming soon